# Let's test out a zero-shot classification model on the data as is

This notebook is the next step after scrapping posts and comments from the subreddit r/AITA and analysing the data collected: <br />
https://github.com/Nico404/scrap_reddit <br />
https://github.com/Nico404/AITA_data_exploration_and_ML/blob/master/AITA_data_exploration.ipynb

In [12]:
import pandas as pd
import pickle
from transformers import pipeline


Let's import our data from AITA_data_exploration.ipynb

In [16]:
with open('data/pickled_post_data.pkl', 'rb') as f:
    data = pickle.load(f)

posts_df = pd.DataFrame(data)

print(posts_df.head())


   post_id                                       post_content  \
0  10uxee0  I know this post sounds super petty, but this ...   
1  10ur722  My daughter Bryn F9 is going on a trip to a ne...   
2  10upxdd  Alright so my son (17) has weekly therapy appo...   
3  10v2vra  We live three blocks away from my parents and ...   
4  10ung90  My daughter (16) and I have gotten into a mass...   

                                          post_title  \
0  AITA for telling my boyfriend I'll shave my le...   
1  AITA for pulling my daughter from a waterpark ...   
2  AITA for not letting an elderly woman have my ...   
3  AITA for taking my kids to my parents house to...   
4  AITA for calling my daughter a selfish insecur...   

                                     comment_results  
0  {'NTA': 31, 'YWBTA': 0, 'YWNBTA': 0, 'ESH': 0,...  
1  {'NTA': 14, 'YWBTA': 0, 'YWNBTA': 0, 'ESH': 0,...  
2  {'NTA': 27, 'YWBTA': 0, 'YWNBTA': 0, 'ESH': 0,...  
3  {'NTA': 63, 'INFO': 1, 'YWBTA': 0, 'YWNBTA': 0...

In [20]:
#let's setup our transformer label candidates and load our model 
candidate_labels = ["NTA", "YTA", "ESH", "NAH", "INFO", "YWBTA", "YWNBTA"]
pipe = pipeline(model="facebook/bart-large-mnli")



In [34]:
all_posts_results = {}
for i, row in posts_df.iterrows():
    post_results = {}
    post_content_prediction = pipe(row['post_content'], candidate_labels)
    for label, score in zip(post_content_prediction["labels"], post_content_prediction["scores"]):
        post_results[label] = score
        all_posts_results[row['post_id']] = post_results
    break

all_post_title_results = {}
for i, row in posts_df.iterrows():
    post_title_results = {}
    post_title_prediction = pipe(row['post_title'], candidate_labels)
    for label, score in zip(post_title_prediction["labels"], post_title_prediction["scores"]):
        post_title_results[label] = score
        all_post_title_results[row['post_id']] = post_title_results
    break

all_comment_results = {}
for i, row in posts_df.iterrows():
    all_comment_results[row['post_id']] = row["comment_results"]
    break

print("post content", all_posts_results)
print("post title", all_post_title_results)
print("comment results", all_comment_results)


post content {'10uxee0': {'ESH': 0, 'INFO': 0, 'NAH': 0, 'YTA': 0, 'YWNBTA': 0, 'YWBTA': 0, 'NTA': 0}}
post title {'10uxee0': {'INFO': 0, 'YTA': 0, 'NTA': 0, 'ESH': 0, 'NAH': 0, 'YWBTA': 0, 'YWNBTA': 0}}
comment results {'10uxee0': {'NTA': 31, 'YWBTA': 0, 'YWNBTA': 0, 'ESH': 0, 'NAH': 0, 'INFO': 0, 'YTA': 0}}
